In [58]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 9: Marble Mania ---</h2><p>You talk to the Elves while you wait for your navigation system to <span title="Do you have any idea how long it takes to load navigation data for all of time and space?!">initialize</span>. To pass the time, they introduce you to their favorite <a href="https://en.wikipedia.org/wiki/Marble_(toy)">marble</a> game.</p>
<p>The Elves play this game by taking turns arranging the marbles in a <em>circle</em> according to very particular rules. The marbles are numbered starting with <code>0</code> and increasing by <code>1</code> until every marble has a number.</p>
<p>First, the marble numbered <code>0</code> is placed in the circle. At this point, while it contains only a single marble, it is still a circle: the marble is both clockwise from itself and counter-clockwise from itself. This marble is designated the <em>current marble</em>.</p>
<p>Then, each Elf takes a turn placing the <em>lowest-numbered remaining marble</em> into the circle between the marbles that are <code>1</code> and <code>2</code> marbles <em>clockwise</em> of the current marble. (When the circle is large enough, this means that there is one marble between the marble that was just placed and the current marble.) The marble that was just placed then becomes the <em>current marble</em>.</p>
<p>However, if the marble that is about to be placed has a number which is a multiple of <code>23</code>, <em>something entirely different happens</em>. First, the current player keeps the marble they would have placed, adding it to their <em>score</em>. In addition, the marble <code>7</code> marbles <em>counter-clockwise</em> from the current marble is <em>removed</em> from the circle and <em>also</em> added to the current player's score. The marble located immediately <em>clockwise</em> of the marble that was removed becomes the new <em>current marble</em>.</p>
<p>For example, suppose there are 9 players. After the marble with value <code>0</code> is placed in the middle, each player (shown in square brackets) takes a turn. The result of each of those turns would produce circles of marbles like this, where clockwise is to the right and the resulting current marble is in parentheses:</p>
<pre><code>[-] <em>(0)</em>
[1]  0<em> (1)</em>
[2]  0<em> (2)</em> 1 
[3]  0  2  1<em> (3)</em>
[4]  0<em> (4)</em> 2  1  3 
[5]  0  4  2<em> (5)</em> 1  3 
[6]  0  4  2  5  1<em> (6)</em> 3 
[7]  0  4  2  5  1  6  3<em> (7)</em>
[8]  0<em> (8)</em> 4  2  5  1  6  3  7 
[9]  0  8  4<em> (9)</em> 2  5  1  6  3  7 
[1]  0  8  4  9  2<em>(10)</em> 5  1  6  3  7 
[2]  0  8  4  9  2 10  5<em>(11)</em> 1  6  3  7 
[3]  0  8  4  9  2 10  5 11  1<em>(12)</em> 6  3  7 
[4]  0  8  4  9  2 10  5 11  1 12  6<em>(13)</em> 3  7 
[5]  0  8  4  9  2 10  5 11  1 12  6 13  3<em>(14)</em> 7 
[6]  0  8  4  9  2 10  5 11  1 12  6 13  3 14  7<em>(15)</em>
[7]  0<em>(16)</em> 8  4  9  2 10  5 11  1 12  6 13  3 14  7 15 
[8]  0 16  8<em>(17)</em> 4  9  2 10  5 11  1 12  6 13  3 14  7 15 
[9]  0 16  8 17  4<em>(18)</em> 9  2 10  5 11  1 12  6 13  3 14  7 15 
[1]  0 16  8 17  4 18  9<em>(19)</em> 2 10  5 11  1 12  6 13  3 14  7 15 
[2]  0 16  8 17  4 18  9 19  2<em>(20)</em>10  5 11  1 12  6 13  3 14  7 15 
[3]  0 16  8 17  4 18  9 19  2 20 10<em>(21)</em> 5 11  1 12  6 13  3 14  7 15 
[4]  0 16  8 17  4 18  9 19  2 20 10 21  5<em>(22)</em>11  1 12  6 13  3 14  7 15 
[5]  0 16  8 17  4 18<em>(19)</em> 2 20 10 21  5 22 11  1 12  6 13  3 14  7 15 
[6]  0 16  8 17  4 18 19  2<em>(24)</em>20 10 21  5 22 11  1 12  6 13  3 14  7 15 
[7]  0 16  8 17  4 18 19  2 24 20<em>(25)</em>10 21  5 22 11  1 12  6 13  3 14  7 15
</code></pre>
<p>The goal is to be the <em>player with the highest score</em> after the last marble is used up. Assuming the example above ends after the marble numbered <code>25</code>, the winning score is <code>23+9=<em>32</em></code> (because player 5 kept marble <code>23</code> and removed marble <code>9</code>, while no other player got any points in this very short example game).</p>
<p>Here are a few more examples:</p>
<ul>
<li><code>10</code> players; last marble is worth <code>1618</code> points: high score is <em><code>8317</code></em></li>
<li><code>13</code> players; last marble is worth <code>7999</code> points: high score is <em><code>146373</code></em></li>
<li><code>17</code> players; last marble is worth <code>1104</code> points: high score is <em><code>2764</code></em></li>
<li><code>21</code> players; last marble is worth <code>6111</code> points: high score is <em><code>54718</code></em></li>
<li><code>30</code> players; last marble is worth <code>5807</code> points: high score is <em><code>37305</code></em></li>
</ul>
<p><em>What is the winning Elf's score?</em></p>
</article>


In [59]:
from itertools import cycle
from re import findall


def highscore(game: str) -> int:
    players, last_marble = map(int, findall(r"\d+", game))

    scores = defaultdict(int)
    index, marble = 0, 0
    circle = [0]

    for player in cycle(range(1, players + 1)):
        marble += 1

        if marble % 23 == 0:
            index = (index - 7) % len(circle)
            scores[player] += marble + circle.pop(index)
        else:
            index = (index + 1) % len(circle) + 1
            circle.insert(index, marble)

        if marble >= last_marble:
            return max(scores.values())


tests = [
    {
        "name": "Example 1",
        "game": "9 players; last marble is worth 25 points",
        "expected": 32,
    },
    {
        "name": "Example 2",
        "game": "10 players; last marble is worth 1618 points",
        "expected": 8317,
    },
    {
        "name": "Example 3",
        "game": "13 players; last marble is worth 7999 points",
        "expected": 146373,
    },
    {
        "name": "Example 4",
        "game": "17 players; last marble is worth 1104 points",
        "expected": 2764,
    },
    {
        "name": "Example 5",
        "game": "21 players; last marble is worth 6111 points",
        "expected": 54718,
    },
    {
        "name": "Example 6",
        "game": "30 players; last marble is worth 5807 points",
        "expected": 37305,
    },
]


run_tests_params(highscore, tests)


Test Example 1 passed, for highscore.
Test Example 2 passed, for highscore.
Test Example 3 passed, for highscore.
Test Example 4 passed, for highscore.
Test Example 5 passed, for highscore.
Test Example 6 passed, for highscore.
Success


In [60]:
puzzle = "405 players; last marble is worth 70953 points"

print(f"Part I: {highscore(puzzle)}")

Part I: 422980


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>422980</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Amused by the speed of your answer, the Elves are curious:</p>
<p><em>What would the new winning Elf's score be if the number of the last marble were 100 times larger?</em></p>
</article>

</main>


In [61]:
from collections import deque
from itertools import cycle
from re import findall


def highscore_II(game: str) -> int:
    # replace normal list by a deque (double ended linked list)
    players, last_marble = map(int, findall(r"\d+", game))

    scores = defaultdict(int)
    marble = 0
    circle = deque([0])

    for player in cycle(range(1, players + 1)):
        marble += 1

        if marble % 23 == 0:
            circle.rotate(7)
            scores[player] += marble + circle.pop()
            circle.rotate(-1)
        else:
            circle.rotate(-1)
            circle.append(marble)

        if marble >= last_marble:
            return max(scores.values())


run_tests_params(highscore_II, tests)


Test Example 1 passed, for highscore_II.
Test Example 2 passed, for highscore_II.
Test Example 3 passed, for highscore_II.
Test Example 4 passed, for highscore_II.
Test Example 5 passed, for highscore_II.
Test Example 6 passed, for highscore_II.
Success


In [62]:
puzzle = "405 players; last marble is worth 7095300 points"

print(f"Part II: {highscore_II(puzzle)}")

Part II: 3552041936


<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>3552041936</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
